In [ ]:
from langgraph.graph import StateGraph , START , END
from langchain_huggingface import ChatHuggingFace , HuggingFaceEndpoint , HuggingFacePipeline
from langchain_anthropic import ChatAnthropic , Anthropic
import os
from typing import TypedDict
import getpass

api_key = os.getenv("GOOGLE_API_KEY")

llm = HuggingFaceEndpoint(
    repo_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

In [125]:
class blog(TypedDict):
    title: str
    outline : str
    content : str

In [126]:
def createoutline(state: blog) -> blog:
    f = state.get("f", 1)  # Default to 1 if not present
    if f == 1:
        print("Step 1: Creating outline...")
        title = state["title"]
        outline = model.invoke(f"Generate an outline for a blog titled: {title}").content
        state["outline"] = outline
        state["f"] = 2  # Move to next step
        print("Outline created:\n", outline)
        f = 2
    else:
        print("Skipping createoutline. Current f:", f)
    return state

In [127]:
def create_blog(state: blog) -> blog:
    f = state.get("f", 0)
    if f == 2:
        print("Step 2: Creating blog...")
        title = state["title"]
        outline = state["outline"]
        content = model.invoke(f"Write a blog on the title: {title} using the outline: {outline}").content
        state["content"] = content
        state["f"] = 3  # Done
        print("Blog created:\n", content[:500], "...\n")
        print(state.keys())
    else:
        print("Skipping create_blog. Current f:", f)
    return state



In [128]:
graph = StateGraph(blog)

graph.add_node("createoutline" , createoutline)
graph.add_node("create_blog", create_blog)


# Add direct edge
graph.add_edge(START, 'createoutline')
graph.add_edge('createoutline', 'create_blog')
graph.add_edge('create_blog', END)


workflow = graph.compile()

In [129]:
intial_state = {'title': 'Rise of AI in India'}

final_state = workflow.invoke(intial_state)

print(final_state)

Step 1: Creating outline...


TypeError: "Could not resolve authentication method. Expected either api_key or auth_token to be set. Or for one of the `X-Api-Key` or `Authorization` headers to be explicitly omitted"

In [111]:
print(final_state.keys())

dict_keys(['title', 'outline'])
